In [1]:
!nvidia-smi

Wed Nov  9 03:56:30 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [4]:
!git clone https://github.com/huynguyenanh2000/ZALOAI.git

Cloning into 'ZALOAI'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 22 (delta 3), reused 19 (delta 3), pack-reused 0
Unpacking objects: 100% (22/22), done.


In [5]:
%cd ZALOAI

/content/drive/MyDrive/ZALOAI


In [6]:
!wget https://dl-challenge.zalo.ai/liveness-detection/train.zip
!unzip train.zip

--2022-11-09 03:59:30--  https://dl-challenge.zalo.ai/liveness-detection/train.zip
Resolving dl-challenge.zalo.ai (dl-challenge.zalo.ai)... 49.213.78.231
Connecting to dl-challenge.zalo.ai (dl-challenge.zalo.ai)|49.213.78.231|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1964055614 (1.8G) [application/zip]
Saving to: ‘train.zip’

train.zip           100%[===================>]   1.83G  11.5MB/s    in 2m 49s  

2022-11-09 04:02:21 (11.1 MB/s) - ‘train.zip’ saved [1964055614/1964055614]

Archive:  train.zip
   creating: train/
  inflating: train/label.csv         
   creating: train/videos/
  inflating: train/videos/471.mp4    
  inflating: train/videos/472.mp4    
  inflating: train/videos/473.mp4    
  inflating: train/videos/477.mp4    
  inflating: train/videos/478.mp4    
  inflating: train/videos/483.mp4    
  inflating: train/videos/480.mp4    
  inflating: train/videos/485.mp4    
  inflating: train/videos/489.mp4    
  inflating: train/videos/490.mp4  

In [7]:
!python3 make_dataset.py --root train --dest raw_data --skip 25

100% 1168/1168 [24:14<00:00,  1.25s/it]


In [8]:
from pathlib import Path
import os
import shutil
from glob import glob
from sklearn.model_selection import train_test_split

In [9]:
all_paths = glob('raw_data/*/*')
len(all_paths)

7659

In [10]:
import random

train_size = 0.8
n = train_size * len(all_paths)
random.shuffle(all_paths)
train_paths = all_paths[:int(n)]
val_paths = all_paths[int(n):]

In [11]:
org_root = 'datasets/'

In [12]:
!mkdir -p datasets/train/0
!mkdir -p datasets/train/1
!mkdir -p datasets/val/0
!mkdir -p datasets/val/1

In [13]:
for path in train_paths:
    cls = path.split(os.sep)[-2]
    name = os.path.basename(path)
    dst_org = os.path.join(org_root, 'train', cls, name)
    shutil.copyfile(path, dst_org)

In [14]:
for path in val_paths:
    cls = path.split(os.sep)[-2]
    name = os.path.basename(path)
    dst_org = os.path.join(org_root, 'val', cls, name)
    shutil.copyfile(path, dst_org)

In [15]:
import pandas as pd 
import os 

train_list = []
test_list = []

for file in os.listdir('/content/drive/MyDrive/ZALOAI/datasets/train/0/'):
    link = '/content/drive/MyDrive/ZALOAI/datasets/train/0/'+file
    train_list.append([link, 0])

for file in os.listdir('/content/drive/MyDrive/ZALOAI/datasets/train/1/'):
    link = '/content/drive/MyDrive/ZALOAI/datasets/train/1/'+file
    train_list.append([link, 1])

for file in os.listdir('/content/drive/MyDrive/ZALOAI/datasets/val/0/'):
    link = '/content/drive/MyDrive/ZALOAI/datasets/val/0/'+file
    test_list.append([link, 0])

for file in os.listdir('/content/drive/MyDrive/ZALOAI/datasets/val/1/'):
    link = '/content/drive/MyDrive/ZALOAI/datasets/val/1/'+file
    test_list.append([link, 1])

train_df = pd.DataFrame(train_list, columns=['name', 'label'])
test_df = pd.DataFrame(test_list, columns=['name', 'label'])


train_df.to_csv('./train_data.csv')
test_df.to_csv('./test_data.csv')

In [45]:
!python3 Train.py

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=DenseNet161_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet161_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
cuda
Training Beginning


Epoch (1/5)
----------------------------------
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
Loss

In [43]:
!wget https://dl-challenge.zalo.ai/liveness-detection/public_test.zip
!unzip public_test.zip

--2022-11-09 08:32:09--  https://dl-challenge.zalo.ai/liveness-detection/public_test.zip
Resolving dl-challenge.zalo.ai (dl-challenge.zalo.ai)... 49.213.78.231
Connecting to dl-challenge.zalo.ai (dl-challenge.zalo.ai)|49.213.78.231|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 652910436 (623M) [application/zip]
Saving to: ‘public_test.zip.1’

public_test.zip.1   100%[===================>] 622.66M  11.3MB/s    in 62s     

2022-11-09 08:33:12 (10.1 MB/s) - ‘public_test.zip.1’ saved [652910436/652910436]

Archive:  public_test.zip
replace public/videos/474.mp4? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [44]:
!python3 Test.py --files /content/drive/MyDrive/ZALOAI/public/videos/


/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=DenseNet161_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet161_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
1: 474.mp4
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1960: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
2: 479.mp4
3: 482.mp4
4: 486.mp4
5: 488.mp4
6: 500.mp4
7: 496.mp4
